In [1]:
import vrep
import time
import numpy as np
from scipy import linalg
import math

# Forward Kinematics

In [55]:
def skew(m):
    return np.array([[0 ,-m[2], m[1]],[m[2], 0, -m[0]], [-m[1], m[0], 0]])


In [54]:
def revolute(a,q):
    ab = skew(a)
    bottom = -1*np.dot(ab,q)
    return np.array([[a[0]],[a[1]],[a[2]],[bottom[0]],[bottom[1]],[bottom[2]]])

In [53]:
def brack6(a):
    ws = a[:3]
    vs = a[3:]
    wb = skew(ws)
    ans = np.array([[wb[0,0], wb[0,1], wb[0,2], vs[0]],[wb[1,0], wb[1,1], wb[1,2], vs[1]],[wb[2,0], wb[2,1], wb[2,2], vs[2]], [0,0,0,0]])
    return ans

In [52]:
# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :     
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
     
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([x, y, z])

In [51]:
def eulerAnglesToRotationMatrix(theta) :
     
    R_x = np.array([[1,         0,                  0                   ],
                    [0,         math.cos(theta[0]), -math.sin(theta[0]) ],
                    [0,         math.sin(theta[0]), math.cos(theta[0])  ]
                    ])
         
         
                     
    R_y = np.array([[math.cos(theta[1]),    0,      math.sin(theta[1])  ],
                    [0,                     1,      0                   ],
                    [-math.sin(theta[1]),   0,      math.cos(theta[1])  ]
                    ])
                 
    R_z = np.array([[math.cos(theta[2]),    -math.sin(theta[2]),    0],
                    [math.sin(theta[2]),    math.cos(theta[2]),     0],
                    [0,                     0,                      1]
                    ])
                     
                     
    R = np.dot(R_z, np.dot( R_y, R_x ))
 
    return R

In [60]:
def forward_kinematics(theta):
    
    a0 = np.array([0,0,1])
    q0 = np.array([0,0,0.1045])
    
    a1 = np.array([-1,0,0])
    q1 = np.array([-0.115,0,0.1089])
    
    a2 = np.array([-1,0,0])
    q2 = np.array([-0.115,0,0.3525])
    
    a3 = np.array([-1,0,0]) #Not bad here
    q3 = np.array([-0.115,0,0.5658])
    
    a4 = np.array([0,0,1]) #this joint also doesn't look bad
    q4 = np.array([-0.112,0,0.65])
    
    a5 = np.array([-1,0,0])  #this doesn't look bad, but hard to tell with this joint
    q5 = np.array([-0.11,0,0.6511])
    
    
    M = np.array([[0,0,-1, -0.1940], [0,1,0,0], [1,0,0,0.6511], [0,0,0,1]])
    
    axis = [a0,a1,a2,a3,a4,a5]
    point = [q0,q1,q2,q3,q4,q5]
    
    screw = [revolute(axis[a],point[a]) for a in range(len(axis))]
    
    exp = [linalg.expm(brack6(screw[s])*theta[s]) for s in range (len(screw))]
    
    ex_multiplied = np.identity(4)
    for i in exp:
        ex_multiplied = ex_multiplied.dot(i)
    T = ex_multiplied.dot(M)
    return T

In [56]:
a = np.array([1,2,3])
q = np.array([4,5,6])
rev = revolute(a,q)

theta = np.array([1,1,1,1,1,1])

In [61]:
vrep.simxFinish(-1)

# Connect to V-REP (raise exception on failure)
clientID = vrep.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
if clientID == -1:
    raise Exception('Failed connecting to remote API server')
    

# Start simulation
vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)

# Wait two seconds
time.sleep(2)


result, joint_one_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint1', vrep.simx_opmode_blocking)
result, joint_two_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint2', vrep.simx_opmode_blocking)
result, joint_three_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint3', vrep.simx_opmode_blocking)
result, joint_four_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint4', vrep.simx_opmode_blocking)
result, joint_five_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint5', vrep.simx_opmode_blocking)
result, joint_six_handle = vrep.simxGetObjectHandle(clientID, 'UR3_joint6', vrep.simx_opmode_blocking)
result, suction_handle = vrep.simxGetObjectHandle(clientID, 'UR3_connection', vrep.simx_opmode_blocking)

result, dummyhandle=vrep.simxCreateDummy(clientID,.04,(255,0,0),vrep.simx_opmode_blocking)

joint_angles=(-np.pi,0,np.pi/4,0,np.pi/4,0)

T = forward_kinematics(joint_angles)
R = T[:3,:3]
p = T[:3,3]
euler = rotationMatrixToEulerAngles(R)

vrep.simxSetObjectPosition(clientID,dummyhandle,-1,(p[0],p[1],p[2]),vrep.simx_opmode_oneshot)
vrep.simxSetObjectOrientation(clientID,dummyhandle,-1,(euler[0],euler[1],euler[2]),vrep.simx_opmode_oneshot)

time.sleep(2)
vrep.simxPauseCommunication(clientID,True)
vrep.simxSetJointTargetPosition(clientID, joint_one_handle, -np.pi, vrep.simx_opmode_oneshot)
vrep.simxSetJointTargetPosition(clientID, joint_two_handle, 0, vrep.simx_opmode_oneshot)
vrep.simxSetJointTargetPosition(clientID, joint_three_handle, np.pi/4, vrep.simx_opmode_oneshot)
vrep.simxSetJointTargetPosition(clientID, joint_four_handle, 0, vrep.simx_opmode_oneshot)
vrep.simxSetJointTargetPosition(clientID, joint_five_handle, np.pi/4, vrep.simx_opmode_oneshot)
vrep.simxSetJointTargetPosition(clientID, joint_six_handle, 0, vrep.simx_opmode_oneshot)
vrep.simxPauseCommunication(clientID,False)
    
time.sleep(5)
# Stop simulation
vrep.simxStopSimulation(clientID, vrep.simx_opmode_oneshot)

# Before closing the connection to V-REP, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
vrep.simxGetPingTime(clientID)

# Close the connection to V-REP
vrep.simxFinish(clientID)
